In [1]:
import argparse
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import json
import itertools
import plotly.graph_objects as go

In [32]:
input_datafolder = Path("../output/apple_swift")

data_path = input_datafolder / "data.jsonl"
commits_path = input_datafolder / "commits.jsonl"
repo_info_path = input_datafolder / "repo_info.json"
all_commits_path = input_datafolder / "all_commits.csv"

all_commits = pd.read_csv(all_commits_path)
all_commit_ids = all_commits["commit_id"].tolist()

with open(repo_info_path, "r") as json_file:
    repo_info: dict = json.load(json_file)
df = pd.read_json(data_path, lines=True)
df_commit = pd.read_json(commits_path, lines=True).set_index("commit_id")

df_commit["max_file_depth"] = df_commit["total_files_per_level"].apply(
    lambda lst: len(lst)-len(list(itertools.takewhile(lambda x: x == 0, reversed(lst))))
)
# Combine the datasets
df = df.join(df_commit, on="commit_id")

# Sort both by commit time
#df = df.sort_values("commit_time", ascending=True)
df_commit = df_commit.sort_values("commit_time", ascending=True)
max_file_depth = repo_info["max_file_depth"]


In [36]:
df_commit["max_depth_change"] = df_commit["max_file_depth"].diff()

In [37]:
df_commit[df_commit["n_added"] > 0].groupby("max_depth_change").count()

,commit_time,message,n_added,n_deleted,n_moved,n_renamed,n_struc_changes,total_files_per_level,max_file_depth,total_files,diff_depth
max_depth_change,,,,,,,,,,,
0.0,10861,10861,10861,10861,10861,10861,10861,10861,10861,10861,10861
1.0,3,3,3,3,3,3,3,3,3,3,3
2.0,2,2,2,2,2,2,2,2,2,2,2
5.0,1,1,1,1,1,1,1,1,1,1,1
